In [1]:
pip install open3d==0.10.0

Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy as np
import time
import open3d
import pandas as pd
import matplotlib.pyplot as plt
import sklearn 

In [23]:
pcd=open3d.io.read_point_cloud("C:\\Users\\YASHITHA\\Downloads\\lidar-object-detection-master\\lidar-object-detection-master\\data\\pcd\\data_1\\0000000004.pcd")
open3d.visualization.draw_geometries([pcd])

In [4]:
pwd

'C:\\Users\\YASHITHA'

In [9]:
print("points before downsampling",len(pcd.points))
downpcd=pcd.voxel_down_sample(voxel_size=0.2)
print("points after downsampling",len(downpcd.points))
open3d.visualization.draw_geometries([downpcd])

points before downsampling 115151
points after downsampling 22978


In [10]:
t1=time.time()
_, inliers=downpcd.segment_plane(distance_threshold=0.3,ransac_n=3,num_iterations=250)
inlier_cloud=downpcd.select_by_index(inliers)
outlier_cloud=downpcd.select_by_index(inliers,invert=True)
inlier_cloud.paint_uniform_color([1,0,0])
outlier_cloud.paint_uniform_color([0,0,1])
open3d.visualization.draw_geometries([inlier_cloud,outlier_cloud])
t2=time.time()
print(f"time taken: {t2-t1}")


time taken: 4.498650550842285


In [11]:
with open3d.utility.VerbosityContextManager(open3d.utility.VerbosityLevel.Debug)as cm:
    labels=np.array(outlier_cloud.cluster_dbscan(eps=0.5,min_points=5,print_progress=True))
#class sklearn.cluster.DBSCAN(eps=0.5, *, min_samples=5, metric='euclidean', metric_params=None, algorithm='auto', leaf_size=30, p=None, n_jobs=None)
max_label=labels.max()
max_label=max(labels)
print(f"point cloud has {max_label+1} clusters")
colors=plt.get_cmap("tab20")(labels/(max_label if max_label > 0 else 1))
colors[labels<0]=0
outlier_cloud.colors=open3d.utility.Vector3dVector(colors[:,:3])
open3d.visualization.draw_geometries([outlier_cloud])

[Open3D DEBUG] Precompute Neighbours
[Open3D DEBUG] Done Precompute Neighbours
[Open3D DEBUG] Compute Clusters
[Open3D DEBUG] Done Compute Clusters: 144
point cloud has 144 clusters


In [12]:
pcd_tree=open3d.geometry.KDTreeFlann(outlier_cloud)
print("paint the 15th point red")
outlier_cloud.colors[15]=[1,0,0]
print("find its 20 nearest points,paint blue.")
[k,idx,_]=pcd_tree.search_knn_vector_3d(outlier_cloud.points[15],20)
np.asarray(outlier_cloud.colors)[idx[1:],:]=[0,0,1]
open3d.visualization.draw_geometries([outlier_cloud])

paint the 15th point red
find its 20 nearest points,paint blue.


In [13]:
obbs=[]
indexes=pd.Series(range(len(labels))).groupby(labels,sort=False).apply(list).tolist()

In [14]:
Max_points=1000
Min_points=20
clusters=[]
for i in range(0,len(indexes)):
  nb_pts=len(outlier_cloud.select_by_index(indexes[i]).points)
  if (nb_pts>Min_points and nb_pts<Max_points):
    sub_cloud=outlier_cloud.select_by_index(indexes[i])
    clusters.append(sub_cloud)
    print(np.asarray([sub_cloud.points]))
    obb=sub_cloud.get_axis_aligned_bounding_box()
    obb.color=[0,0,1]
    obbs.append(obb)

[[[ 1.27690001e+01 -2.22649994e+01  4.67000008e-01]
  [ 1.26420002e+01 -2.21669998e+01  9.10000026e-01]
  [ 1.23500004e+01 -2.27919998e+01  6.01999998e-01]
  [ 1.25930004e+01 -2.24769993e+01  6.00000024e-01]
  [ 1.19776669e+01 -2.33100001e+01 -9.62000012e-01]
  [ 1.25704999e+01 -2.24504995e+01  9.97500002e-01]
  [ 1.28050003e+01 -2.22360001e+01  7.92999983e-01]
  [ 1.24282501e+01 -2.24565001e+01  8.53249997e-01]
  [ 1.19320002e+01 -2.29134998e+01  7.96999991e-01]
  [ 1.29130001e+01 -2.22350006e+01  9.15000021e-01]
  [ 1.25799999e+01 -2.25049992e+01  4.67999995e-01]
  [ 1.25685000e+01 -2.22825003e+01  1.07400000e+00]
  [ 1.25530005e+01 -2.18659992e+01  2.86000013e-01]
  [ 1.24370003e+01 -2.28349991e+01  4.69999999e-01]
  [ 1.25570002e+01 -2.28640003e+01  2.89000005e-01]
  [ 1.23823334e+01 -2.31066672e+01 -9.62333341e-01]
  [ 1.23683332e+01 -2.26696663e+01  1.08233333e+00]
  [ 1.24569998e+01 -2.24950008e+01  1.07900000e+00]
  [ 1.18190002e+01 -2.33610001e+01 -9.61000025e-01]
  [ 1.241099

  [ -8.87166659   4.14833323  -1.52633333]]]
[[[-14.16799974   7.92500019   0.75825001]
  [-14.13114262   8.31728581   0.421     ]
  [-13.98349988   9.51875007   0.0205    ]
  [-14.30599976   7.50099993   0.755     ]
  [-14.0060002    9.69650014   0.22016667]
  [-13.61200015   9.64000003  -1.57733333]
  [-14.04899979   9.11466662   0.77533334]
  [-14.03080006   8.94439983   0.63440001]
  [-14.21425009   8.88649988   0.4195    ]
  [-14.17199993   7.71675003   0.75475   ]
  [-13.60050011   9.69050026  -0.231     ]
  [-14.03383319   8.92250013   0.418     ]
  [-13.5795002    9.71700001   0.77399999]
  [-13.7579999    9.71399975   0.77849999]
  [-14.03349972   9.6789999    0.78550002]
  [-14.03866673   9.52066676   0.78266666]
  [-14.04425001   9.32500029   0.77949999]
  [-14.05466684   8.95233345   0.77233334]
  [-14.20149994   8.89949989   0.7755    ]
  [-14.16550016   8.48000002   0.76750001]
  [-14.1647501    8.30699968   0.76425   ]
  [-14.01133315   9.50616678   0.22      ]
  [-14.15

  [-20.04800034  -4.78700018  -1.57099998]]]
[[[-2.91450000e+00  2.02734995e+01 -3.90000001e-01]
  [-2.81200004e+00  2.14020004e+01 -9.70000029e-02]
  [-2.58800006e+00  2.13959999e+01  3.59999985e-02]
  [-2.82000005e+00  2.02632504e+01  1.32500008e-02]
  [-2.42066669e+00  2.14216671e+01  3.59999985e-02]
  [-2.81900001e+00  2.09489994e+01 -9.09999982e-02]
  [-2.89299989e+00  1.99470005e+01  4.69999984e-02]
  [-2.84100008e+00  2.06399994e+01  1.73999995e-01]
  [-2.66866660e+00  2.14913336e+01 -1.48333331e-01]
  [-1.77600002e+00  2.14339994e+01  3.59999985e-02]
  [-2.86100006e+00  2.03139992e+01  1.74999997e-01]
  [-1.22466667e+00  2.14539992e+01  3.66666665e-02]
  [-2.85700011e+00  2.09990005e+01  1.73999995e-01]
  [-1.97933332e+00  2.14263331e+01  3.59999985e-02]
  [-2.91499996e+00  2.00990009e+01 -2.21000001e-01]
  [-2.82100010e+00  2.07360001e+01 -3.40999991e-01]
  [-2.79050004e+00  2.04514999e+01 -5.92000008e-01]
  [-1.03366667e+00  2.14560000e+01  3.70000005e-02]
  [-2.41599997e+00 

In [16]:
list_of_visuals=[]
list_of_visuals.append(outlier_cloud)
list_of_visuals.extend(obbs)
list_of_visuals.append(inlier_cloud)
open3d.visualization.draw_geometries(list_of_visuals)

In [17]:
print(len(list_of_visuals))

61


In [18]:
print(np.asarray([clusters]))

[[geometry::PointCloud with 57 points.
  geometry::PointCloud with 78 points.
  geometry::PointCloud with 864 points.
  geometry::PointCloud with 72 points.
  geometry::PointCloud with 122 points.
  geometry::PointCloud with 83 points.
  geometry::PointCloud with 37 points.
  geometry::PointCloud with 157 points.
  geometry::PointCloud with 278 points.
  geometry::PointCloud with 533 points.
  geometry::PointCloud with 94 points.
  geometry::PointCloud with 101 points.
  geometry::PointCloud with 279 points.
  geometry::PointCloud with 28 points.
  geometry::PointCloud with 133 points.
  geometry::PointCloud with 460 points.
  geometry::PointCloud with 200 points.
  geometry::PointCloud with 29 points.
  geometry::PointCloud with 107 points.
  geometry::PointCloud with 174 points.
  geometry::PointCloud with 262 points.
  geometry::PointCloud with 44 points.
  geometry::PointCloud with 297 points.
  geometry::PointCloud with 148 points.
  geometry::PointCloud with 146 points.
  geometr

In [19]:
arr = []
for i in np.asarray([clusters][0]):
    arr.append(len(i.points))
denom = max(arr)

In [20]:
open3d.visualization.draw_geometries(list_of_visuals)

In [21]:
def f(x):
    if x>0.8:
        return "Wall"
    if x>0.7:
        return "Truck"
    if x>0.5:
        return "Van"
    if x>0.2 and x<0.4:
        return "Car"   
    else:
        return ""

In [22]:
for i in np.asarray([clusters][0]):
    print (i,  f(len(i.points)/denom))

geometry::PointCloud with 57 points. 
geometry::PointCloud with 78 points. 
geometry::PointCloud with 864 points. Wall
geometry::PointCloud with 72 points. 
geometry::PointCloud with 122 points. 
geometry::PointCloud with 83 points. 
geometry::PointCloud with 37 points. 
geometry::PointCloud with 157 points. 
geometry::PointCloud with 278 points. Car
geometry::PointCloud with 533 points. Van
geometry::PointCloud with 94 points. 
geometry::PointCloud with 101 points. 
geometry::PointCloud with 279 points. Car
geometry::PointCloud with 28 points. 
geometry::PointCloud with 133 points. 
geometry::PointCloud with 460 points. Van
geometry::PointCloud with 200 points. Car
geometry::PointCloud with 29 points. 
geometry::PointCloud with 107 points. 
geometry::PointCloud with 174 points. Car
geometry::PointCloud with 262 points. Car
geometry::PointCloud with 44 points. 
geometry::PointCloud with 297 points. Car
geometry::PointCloud with 148 points. 
geometry::PointCloud with 146 points. 
geomet